Pixeltests School Data Science

*Unit 2, Sprint 2, Module 3*

---
<p style="padding: 10px; border: 2px solid red;">
    <b>Before you start:</b> Today is the day you should submit the dataset for your Unit 2 Build Week project. You can review the guidelines and make your submission in the Build Week course for your cohort on Canvas.</p>

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/pixeltests/datasets/main/' #You might not need this, use the data from Kaggle directly!
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Module Project: Hyperparameter Tuning

This sprint, the module projects will focus on creating and improving a model for the Tanazania Water Pump dataset. Your goal is to create a model to predict whether a water pump is functional, non-functional, or needs repair.

Dataset source: [DrivenData.org](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/).

## Directions

- **Task 0:** Enter the [Kaggle](https://www.kaggle.com/t/6169ee7701164d24943c98eda2de9b5e) competition using exactly this link!
The tasks for this project are as follows:

- **Task 1:** Use `wrangle` function to import training and test data.
- **Task 2:** Split training data into feature matrix `X` and target vector `y`.
- **Task 3:** Establish the baseline accuracy score for your dataset.
- **Task 4:** Build `clf_dt`.
- **Task 5:** Build `clf_rf`.
- **Task 6:** Evaluate classifiers using k-fold cross-validation.
- **Task 7:** Tune hyperparameters for best performing classifier.
- **Task 8:** Print out best score and params for model.
- **Task 9:** Create `submission.csv` and upload to Kaggle.

You should limit yourself to the following libraries for this project:

- `category_encoders`
- `matplotlib`
- `pandas`
- `pandas-profiling`
- `sklearn`

# I. Wrangle Data

In [2]:
def wrangle(fm_path, tv_path=None):
    if tv_path:
        df = pd.merge(pd.read_csv(fm_path,
                                  na_values=[0, -2.000000e-08]),
                      pd.read_csv(tv_path)).set_index('id')
    else:
        df = pd.read_csv(fm_path,
                         na_values=[0, -2.000000e-08],
                         index_col='id')
   #df.drop(['id'],axis = 1,inplace = True)
    #df.drop(columns=['id'], inplace=True)
    # Drop constant columns
    df.drop(columns=['recorded_by'], inplace=True)

    # Drop HCCCs
    cutoff = 100
    drop_cols = [col for col in df.select_dtypes('object').columns
                 if df[col].nunique() > cutoff]
    df.drop(columns=drop_cols, inplace=True)

    # Drop duplicate columns
    dupe_cols = [col for col in df.head(15).T.duplicated().index
                 if df.head(15).T.duplicated()[col]]
    df.drop(columns=dupe_cols, inplace=True)

    return df

In [5]:
from google.colab import files
uploaded = files.upload()


Saving test_features.csv to test_features.csv
Saving train_features.csv to train_features.csv
Saving train_labels.csv to train_labels.csv


In [3]:
def preprocess(df):

  df.drop('id',axis = 1,inplace = True)
  #Dropping the recorded by column

  df.drop('recorded_by',axis = 1,inplace = True)

  #Engineering 'pump-age'

  #df['pump_age'] = 2023 - df['construction_year']

  #Dropping the HCC's

  cutoff = 100

  drop_cols = [col for col in df.select_dtypes('object').columns
             if df[col].nunique() > cutoff]
  df.drop(drop_cols , inplace = True , axis = 1)

  #Dropping the duplicate columns

  dupe_cols = [col for col in df.head(15).T.duplicated().index
             if df.head(15).T.duplicated()[col]]
  df.drop(dupe_cols , inplace = True , axis = 1)

  return df

In [6]:
import pandas as pd
import io

X_test1 = pd.read_csv(io.BytesIO(uploaded['test_features.csv']))
X_test = preprocess(X_test1)
#X_test = wrangle(io.BytesIO(uploaded['test_features.csv']))
#X_test1.drop('id',axis = 1,inplace = True)
#X_test.shape, X_test1.shape

#X_test1.shape
#X_test.head()
#X_test.drop(['id'], axis=1, inplace=True)
#X_test1.reset_index(drop = True).shape
X_test.head(2)
X_test.columns
#X_test.drop(['id'], axis=1, inplace = True)
X_test.shape

(11880, 29)

In [ ]:
X_test.head(1)

,amount_tsh,gps_height,longitude,latitude,num_private,basin,region,region_code,district_code,population,...,payment,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,0.0,0,32.899425,-3.316937,0,Lake Victoria,Shinyanga,17,3,0,...,never pay,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [ ]:
X_test.columns

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'basin', 'region', 'region_code', 'district_code', 'population',
       'public_meeting', 'scheme_management', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [ ]:
X_test.head(2)

,amount_tsh,gps_height,longitude,latitude,num_private,basin,region,region_code,district_code,population,...,payment,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,0.0,0,32.899425,-3.316937,0,Lake Victoria,Shinyanga,17,3,0,...,never pay,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,hand pump
1,0.0,0,33.454783,-8.755486,0,Lake Rukwa,Mbeya,12,2,0,...,pay monthly,monthly,soft,good,enough,river,river/lake,surface,communal standpipe,communal standpipe


**Task 1:** Using the above `wrangle` function to read `train_features.csv` and `train_labels.csv` into the DataFrame `df`, and `test_features.csv` into the DataFrame `X_test`.

In [10]:
import pandas as pd
#df = wrangle(DATA_PATH+'/waterpumps/train_features.csv', DATA_PATH+'/waterpumps/train_labels.csv')
df1 = pd.read_csv(io.BytesIO(uploaded['train_features.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded['train_labels.csv']))
df = pd.concat([df1, df2], axis=1)

#df = pd.merge(pd.read_csv(DATA_PATH+'/waterpumps/train_features.csv'),
                                  #pd.read_csv( DATA_PATH+'/waterpumps/train_labels.csv'))

df = preprocess(df)
df.shape
#X_test = pd.read_csv(DATA_PATH+'/test_features.csv')
#df.shape, X_test.shape

(47520, 30)

In [ ]:
df.columns

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'basin', 'region', 'region_code', 'district_code', 'population',
       'public_meeting', 'scheme_management', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'status_group'],
      dtype='object')

# II. Split Data

**Task 2:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'status_group'`.

**Note:** You won't need to do a train-test split because you'll use cross-validation instead.

In [11]:
X = df.drop('status_group', axis =1)
y = df['status_group']

X.shape, y.shape

((47520, 29), (47520,))

# III. Establish Baseline

**Task 3:** Since this is a **classification** problem, you should establish a baseline accuracy score. Figure out what is the majority class in `y_train` and what percentage of your training observations it represents.

In [ ]:
baseline_acc = ...
print('Baseline Accuracy Score:', baseline_acc)

# IV. Build Models

**Task 4:** Build a `Pipeline` named `clf_dt`. Your `Pipeline` should include:

- an `OrdinalEncoder` transformer for categorical features.
- a `SimpleImputer` transformer fot missing values.
- a `DecisionTreeClassifier` Predictor.

**Note:** Do not train `clf_dt`. You'll do that in a subsequent task.

In [12]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

#set up a pipeline

clf_dt = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    DecisionTreeClassifier(max_depth = 5, random_state=91)
    )


**Task 5:** Build a `Pipeline` named `clf_rf`. Your `Pipeline` should include:

- an `OrdinalEncoder` transformer for categorical features.
- a `SimpleImputer` transformer fot missing values.
- a `RandomForestClassifier` predictor.

**Note:** Do not train `clf_rf`. You'll do that in a subsequent task.

In [13]:
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

#build a pipeline for a random forest but with an ordinal encoder this time
clf_rf = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_jobs=-1, random_state=25)
)



# V. Check Metrics

**Task 6:** Evaluate the performance of both of your classifiers using k-fold cross-validation.

In [14]:

from sklearn.model_selection import cross_val_score
k = 3
cv_scores_dt = cross_val_score(clf_dt, X, y, cv=k,
                         scoring='accuracy')

cv_scores_rf = cross_val_score(clf_rf, X, y, cv=k,
                         scoring='accuracy')

In [15]:
print('CV scores DecisionTreeClassifier')
print(cv_scores_dt)
print('Mean CV accuracy score:', cv_scores_dt.mean())
print('STD CV accuracy score:', cv_scores_dt.std())

CV scores DecisionTreeClassifier
[0.71092172 0.71546717 0.71073232]
Mean CV accuracy score: 0.7123737373737374
STD CV accuracy score: 0.0021887545231514516


In [16]:
print('CV score RandomForestClassifier')
print(cv_scores_rf)
print('Mean CV accuracy score:', cv_scores_rf.mean())
print('STD CV accuracy score:', cv_scores_rf.std())

CV score RandomForestClassifier
[0.79135101 0.80012626 0.79488636]
Mean CV accuracy score: 0.7954545454545454
STD CV accuracy score: 0.0036049398658296824


# VI. Tune Model

**Task 7:** Choose the best performing of your two models and tune its hyperparameters using a `RandomizedSearchCV` named `model`. Make sure that you include cross-validation and that `n_iter` is set to at least `25`.

**Note:** If you're not sure which hyperparameters to tune, check the notes from today's guided project and the `sklearn` documentation.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
     'randomforestclassifier__max_depth': [10, 15, 35, 80, 90, 100],
    'randomforestclassifier__min_samples_leaf':[2,4,5,6]
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    clf_rf,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder()),
                                             ('simpleimputer',
                                              SimpleImputer(strategy='median')),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(n_jobs=-1,
                                                                     random_state=25))]),
                   n_jobs=-1,
                   param_distributions={'randomforestclassifier__max_depth': [10,
                                                                              15,
                                                                              35,
                                                                              80,
                                                                              90,
                                                                              100],
                                        'randomforestclassifier__min_samples_leaf': [2,
                                                                                     4,
                                                                                     5,
                                                                                     6],
                                        'simpleimputer__strategy': ['mean',
                                                                    'median']},
                   return_train_score=True, scoring='accuracy', verbose=10)

**Task 8:** Print out the best score and best params for `model`.

In [ ]:
best_score = search.best_score_
best_params = search.best_params_

print('Best score for `model`:', best_score)
print('Best params for `model`:', best_params)

Best score for `model`: 0.8107575757575758
Best params for `model`: {'simpleimputer__strategy': 'median', 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__max_depth': 80}


# Communicate Results

**Task 9:** Create a DataFrame `submission` whose index is the same as `X_test` and that has one column `'status_group'` with your predictions. Next, save this DataFrame as a CSV file and upload your submissions to our competition site.

**Note:** Check the `sample_submission.csv` file on the competition website to make sure your submissions follows the same formatting.

In [17]:
clf_rf = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_jobs=-1, random_state=25,min_samples_leaf=2,max_depth=90)
)
clf_rf.fit(X,y)
#predict on test
y_pred= clf_rf.predict(X_test)
y_pred

data = {
    "S.No.": range(1, 1 + len(X_test)),
    "status_group": y_pred
}
submission = pd.DataFrame(data)
#encode status_group
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

submission['status_group'] = le.fit_transform(submission['status_group'] )
submission['status_group']
#submission = pd.DataFrame(y_pred, index=ids)
submission.to_csv('submission-cv.csv', index=False)

Submissions
You selected 0 of 1 submissions to be evaluated for your final leaderboard score. Since you selected less than 1 submission, Kaggle auto-selected up to 1 submissions from among your public best-scoring unselected submissions for evaluation. The evaluated submission with the best Private Score is used for your final score.

Submissions evaluated for final score

0/1

All

Successful

Selected

Errors

Recent
Submission and Description

Private Score

Public Score

Selected

submission-cv (2).csv
Complete (after deadline) · now
0.82531

0.82531

